In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.nn import functional as F
from tqdm.notebook import tqdm
import warnings
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split
from torch import torch
from sklearn.model_selection import train_test_split
warnings.filterwarnings(action='ignore')
%matplotlib inline

# 1. 모델을 불러옵니다.

In [ ]:
from transformers import ElectraModel, ElectraTokenizer
from transformers import ElectraForSequenceClassification, AdamW
from transformers import get_cosine_schedule_with_warmup

model = ElectraForSequenceClassification.from_pretrained('monologg/koelectra-small-v2-discriminator')
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-small-v2-discriminator")

In [ ]:
model.classifier.out_proj =  nn.Sequential( nn.Linear(256, 1),
                                           nn.Sigmoid())

model.load_state_dict(torch.load(f'/content/gdrive/MyDrive/A3_datasets/KoElectra/koelectra_datasets/koelectra_model5/checkpoint_{cate}.pt'))

In [ ]:
device = torch.device("cuda:0")
model.to(device)

# 2. 모두에게 동일하게 적용될 테스트셋을 생성합니다.

In [ ]:
class LoadDataset(Dataset):
    def __init__(self, df, tk):
        self.df = df
        self.tokenizer = tk

    def __len__(self):
        return len(self.df)
  
    def __getitem__(self, idx):
        row = self.df.iloc[idx, :].values
        # target이 없는경우 (즉, 문장만 입력된 경우)
        if len(row) <= 1:
            text = row[0]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=50,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]

            return input_ids, attention_mask     
            
        # target이 있는 경우 (원래 코드)
        else:
            text = row[0]
            y = row[1]

            inputs = self.tokenizer(
                text, 
                return_tensors='pt',
                truncation=True,
                max_length=50,
                pad_to_max_length=True,
                add_special_tokens=True
                )
            
            input_ids = inputs['input_ids'][0]
            attention_mask = inputs['attention_mask'][0]

            return input_ids, attention_mask, y

In [ ]:
# 테스트용 데이터를 불러옵니다.
PATH = '/content/gdrive/MyDrive/A3_datasets/KoElectra/koelectra_datasets/'
cate = '악플욕설'
df = pd.read_csv(PATH + f'{cate}.csv')
df = df[['문장', f'{cate}']]

def dataSplit(dataset, y_label):
  X_train, X_val= train_test_split(dataset, test_size = 0.2, stratify = dataset[y_label], random_state =427)
  return X_train, X_val

X_train, X_test = dataSplit(df, cate)

In [ ]:
test_set = LoadDataset(X_test, tokenizer) # 이 테스트셋은 unsmile.csv로만 전이학습한 모델의 평가데이터와 동일합니다.

# 3. 추가로 전이학습할 카카오 데이터 불러오기

In [ ]:
# 카카오 데이터를 불러옵니다.
PATH = '/content/gdrive/MyDrive/A3_datasets/sudo_labeling/kakao/'
kakao_df = pd.read_csv(PATH + f'{cate}.csv')
kakao_df = kakao_df[['text',f'{cate}']]

X_train, X_valid = dataSplit(kakao_df, cate)

train_set = LoadDataset(X_train, tokenizer)
val_set = LoadDataset(X_valid, tokenizer)

# 4. 카카오 데이터로 전이학습

In [ ]:
device = torch.device("cuda:0")
model.to(device)

In [ ]:
from transformers import get_cosine_schedule_with_warmup, AdamW

epochs = 300 # epochs 증가
batch_size = 64 # batch size 감소
warmup_ratio=0.1
t_total = len(train_set) * epochs
optimizer = AdamW(model.parameters(), lr=1e-5, eps = 1e-8) # lr 1/10으로 변경
train_loader = DataLoader(train_set, batch_size=batch_size)
val_loader = DataLoader(val_set, batch_size=batch_size) # val loader 추가
test_loader = DataLoader(test_set, batch_size = batch_size) # test loader 추가
loss_f = nn.BCEWithLogitsLoss() # loss f 변경

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=1, num_training_steps=t_total)

In [ ]:
# https://github.com/Bjarten/early-stopping-pytorch
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path=f'/content/gdrive/MyDrive/A3_datasets/KoElectra/koelectra_datasets/koelectra_kakao/checkpoint_{cate}_kakao.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [ ]:
early_stopping = EarlyStopping(patience = 7, verbose = True)

In [ ]:
from sklearn.metrics import roc_auc_score

for i in tqdm(range(epochs)):
    train_loss_list = [] # 변수 변경
    val_loss_list = []
    val_score_list = []

    epoch_train_loss = []
    epoch_val_loss = []
    epoch_val_score = []
    # train
    model.train()
    for input_ids_batch, attention_masks_batch, y_batch in train_loader:
        input_ids_batch = input_ids_batch.to(device)
        attention_masks_batch = attention_masks_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(input_ids_batch, attention_mask=attention_masks_batch).logits.reshape(-1)
#        print(y_pred)
        loss = loss_f(y_pred.type(torch.FloatTensor), y_batch.type(torch.FloatTensor))
#        print(loss)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss_list.append(loss.item())

    # validation loss
    model.eval()
    for input_ids_batch_val, attention_masks_batch_val, y_batch_val in val_loader:
        input_ids_batch_val = input_ids_batch_val.to(device)
        attention_masks_batch_val = attention_masks_batch_val.to(device)
        y_batch_val = y_batch_val.to(device)
        y_pred_val = model(input_ids_batch_val, attention_mask = attention_masks_batch_val).logits.reshape(-1)
        loss = loss_f(y_pred_val.type(torch.FloatTensor), y_batch_val.type(torch.FloatTensor))
        val_score = roc_auc_score(y_batch_val.tolist(), y_pred_val.tolist())
        val_loss_list.append(loss.item())
        val_score_list.append(val_score)

    # epoch당 loss 계산 (for early stopping)
    train_loss = np.average(train_loss_list)
    val_loss = np.average(val_loss_list)
    val_score = np.average(val_score_list)

    epoch_train_loss.append(train_loss)
    epoch_val_loss.append(val_loss)
    epoch_val_score.append(val_score)
    epoch_len = len(str(epochs))

    print_msg = (f'[{i:>{epoch_len}}/{epochs:>{epoch_len}}] ' +
                 f'train_loss: {train_loss:.5f} ' +
                 f'valid_loss: {val_loss:.5f} ' +
                 f'valid_score: {val_score:.5f}')

    print(print_msg)
    
    # clear lists to track next epoch
    train_loss_list = []
    val_loss_list = []
    val_score_list = []
    early_stopping(val_loss, model)
    if early_stopping.early_stop:
        print('early stopping')
        break
    
model.load_state_dict(torch.load(f'/content/gdrive/MyDrive/A3_datasets/KoElectra/koelectra_datasets/koelectra_kakao/checkpoint_{cate}_kakao.pt'))

# 5. 스코어 확인하기

In [ ]:
model.eval()
score_list = []
for input_ids_batch, attention_masks_batch, y_batch in tqdm(test_loader):
    input_ids_batch = input_ids_batch.to(device)
    attention_masks_batch = attention_masks_batch.to(device)
    y_batch = y_batch.to(device)
    y_pred = model(input_ids_batch, attention_mask=attention_masks_batch).logits.reshape(-1)
#    print(y_pred)
    try:
        score = roc_auc_score(y_batch.tolist(), y_pred.tolist())
        score_list.append(score)
    except: pass
print('cate : ', cate)
print("epoch roc_auc:", np.mean(score_list))